# Otimização de Markowitz

## Explicação

### Índice Sharpe

Mede a rentabilidade em relação à volatilidade da carteira. É normalmente apresentado junto com a volatilidade líquida e a rentabilidade líquida.

Se considerarmos o CDI como ativo livre de risco, a fórmula fica:

<div style="text-align: left">

$$
\frac{\text{rentabilidade do ativo} - \text{CDI}}{\text{volatilidade do ativo}}
$$

</div>


Em alguns casos, podemos desconsiderar o ativo livre de risco. Por exemplo:

Se o retorno foi 20% e a volatilidade foi 10%, o Sharpe do investimento é 2.

---

### Fórmula canônica

<div style="text-align: left">

$$
\text{Sharpe} = \frac{R_p - R_f}{\sigma_p}
$$

</div>

* \(R_p\) — retorno médio do ativo ou portfólio  
* \(R_f\) — taxa livre de risco (ex.: CDI, T‑bill)  
* \(\sigma_p\) — volatilidade (desvio padrão) dos retornos de \(R_p\)

Essa fórmula mede o **retorno excedente por unidade de risco total**.

---

### Quando se usa \(R_f = 0\)

| Situação | Por quê? |
| --- | --- |
| Exemplos didáticos | Para simplificar a explicação e a matemática. |
| Séries já em *excess return* | O retorno livre de risco já foi subtraído. |
| Frequência diária ou intradiária | A taxa diária do CDI é desprezível. |
| Ambientes com juro ≈ 0% | Comuns após 2008 em EUA, Europa ou Japão. |
| Estratégias *self-financing* | O retorno já é líquido de funding (ex.: market-neutral). |

---

### Exemplo

* Retorno anual do ativo: **20%**  
* CDI anual: **12%**  
* Volatilidade anual: **10%**

<div style="text-align: left">

$$
\text{Sharpe} = \frac{0.20 - 0.12}{0.10} = 0.8
$$

</div>

*Se \(R_f = 0\), o mesmo caso daria Sharpe = 2.*

---

### Como funciona a otimização

O objetivo da otimização de Markowitz é **encontrar a carteira com o maior índice de Sharpe**.

Por quê?  
Porque quanto maior o Sharpe, **maior o retorno esperado para cada unidade de risco** (volatilidade) assumido. Isso significa melhor eficiência na relação risco-retorno.

---


# Código

## Importações e Funções Auxiliares

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import mplcyberpunk
import datetime as dt
import pandas as pd
import scipy.optimize as minimize
import matplotlib.ticker as mticker
from IPython.display import display
from tqdm import tqdm



In [ ]:
def mostrar_top_carteiras(vetor_ordenacao, vetor_retornos_esperados, tabela_pesos, tickers, top_n=10, nome_ordenacao="Índice"):
    """
    Exibe um DataFrame com as top carteiras ordenadas por uma métrica (Sharpe, Sharpe penalizado, etc.).

    Parâmetros:
    - vetor_ordenacao: vetor com a métrica a ser usada para ordenação (ex: vetor_sharpe, vetor_sharpe_penalizado)
    - vetor_retornos_esperados: vetor com retornos logarítmicos esperados anualizados (252 dias)
    - tabela_pesos: matriz com os pesos dos ativos para cada carteira
    - tickers: lista dos nomes dos ativos
    - top_n: número de carteiras a exibir (default = 10)
    - nome_ordenacao: nome da métrica que está sendo usada (só para exibição)
    """
    # Converte log-retorno em retorno aritmético anual
    retornos_anuais_aritmeticos = np.exp(vetor_retornos_esperados) - 1

    # Ordena os índices do vetor de ordenação (maior para menor)
    indices_ordenados = np.argsort(vetor_ordenacao)[::-1]

    # Monta tabela com retornos e pesos
    dados_tabela = []
    for i in indices_ordenados[:top_n]:
        linha = {
            'Retorno Esperado (%)': f'{retornos_anuais_aritmeticos[i] * 100:.2f}',
            nome_ordenacao: f'{vetor_ordenacao[i]:.4f}'
        }
        for j, ticker in enumerate(tickers):
            linha[ticker] = f'{tabela_pesos[i, j] * 100:.2f}%'
        dados_tabela.append(linha)

    df_resultado = pd.DataFrame(dados_tabela)

    print(f"\n📋 Top {top_n} carteiras ordenadas por: {nome_ordenacao}\n")
    return df_resultado


def plot_fronteira_eficiente(volatilidades, retornos, vetor_ordenacao, titulo="Fronteira Eficiente",
                              label_ordenacao="Sharpe", cor_destaque='red', idx_vizinho=None, idx_otimo=None):
    """
    Gera o gráfico da fronteira eficiente com destaque para:
    - melhor carteira segundo a métrica fornecida
    - uma carteira "vizinha" (ex: 900ª melhor)
    """
    # Corrige: converte retornos de logarítmico para aritmético
    retornos_aritmeticos = np.exp(retornos) - 1
    retornos_pct = retornos_aritmeticos * 100
    vols_pct = volatilidades * 100

    # Para a fronteira, ordenar apenas para a linha
    indices_ordenados = np.argsort(vols_pct)
    vols_ordenados = vols_pct[indices_ordenados]
    retornos_ordenados = retornos_pct[indices_ordenados]

    fronteira_vols = []
    fronteira_rets = []
    max_ret = -np.inf
    for vol, ret in zip(vols_ordenados, retornos_ordenados):
        if ret > max_ret:
            fronteira_vols.append(vol)
            fronteira_rets.append(ret)
            max_ret = ret

    if idx_otimo is None:
        idx_otimo = vetor_ordenacao.argmax()

    plt.figure(figsize=(10, 6))
    sc = plt.scatter(vols_pct, retornos_pct, c=vetor_ordenacao, cmap='viridis', s=8, alpha=0.8)
    plt.plot(fronteira_vols, fronteira_rets, color='blue', linewidth=2, label='Fronteira eficiente')
    plt.colorbar(sc, label=label_ordenacao)

    plt.scatter(vols_pct[idx_otimo], retornos_pct[idx_otimo],
                color=cor_destaque, marker='o', s=60, label=f'Máx. {label_ordenacao}')

    if idx_vizinho is not None:
        plt.scatter(vols_pct[idx_vizinho], retornos_pct[idx_vizinho],
                    color='deepskyblue', marker='s', s=60, label=f'Carteira {idx_vizinho}')

    plt.xlabel('Volatilidade esperada')
    plt.ylabel('Retorno esperado')
    plt.title(titulo)
    plt.legend()
    plt.grid(True)

    plt.xticks(np.round(np.linspace(min(vols_pct), max(vols_pct), 6), 1),
               labels=[f'{x:.1f}%' for x in np.linspace(min(vols_pct), max(vols_pct), 6)])
    plt.yticks(np.round(np.linspace(min(retornos_pct), max(retornos_pct), 6), 1),
               labels=[f'{x:.1f}%' for x in np.linspace(min(retornos_pct), max(retornos_pct), 6)])

    plt.tight_layout()
    plt.show()


def plot_fronteira_eficiente_cyberpunk(volatilidades, retornos, vetor_ordenacao, titulo="Fronteira Eficiente",
                              label_ordenacao="Sharpe", cor_destaque='red', idx_vizinho=None, idx_otimo=None):

    plt.style.use("cyberpunk")

    retornos_aritmeticos = np.exp(retornos) - 1
    retornos_pct = retornos_aritmeticos * 100
    vols_pct = volatilidades * 100

    indices_ordenados = np.argsort(vols_pct)
    vols_ordenados = vols_pct[indices_ordenados]
    retornos_ordenados = retornos_pct[indices_ordenados]

    fronteira_vols = []
    fronteira_rets = []
    max_ret = -np.inf
    for vol, ret in zip(vols_ordenados, retornos_ordenados):
        if ret > max_ret:
            fronteira_vols.append(vol)
            fronteira_rets.append(ret)
            max_ret = ret

    if idx_otimo is None:
        idx_otimo = vetor_ordenacao.argmax()

    plt.figure(figsize=(10, 6))
    sc = plt.scatter(vols_pct, retornos_pct, c=vetor_ordenacao, cmap='viridis', s=8, alpha=0.8)
    plt.plot(fronteira_vols, fronteira_rets, color='cyan', linewidth=2, label='Fronteira eficiente')
    plt.colorbar(sc, label=label_ordenacao)

    plt.scatter(vols_pct[idx_otimo], retornos_pct[idx_otimo],
                color=cor_destaque, marker='o', s=60, label=f'Máx. {label_ordenacao}')

    if idx_vizinho is not None:
        plt.scatter(vols_pct[idx_vizinho], retornos_pct[idx_vizinho],
                    color='deepskyblue', marker='s', s=60, label=f'Carteira {idx_vizinho}')

    plt.xlabel('Volatilidade esperada')
    plt.ylabel('Retorno esperado')
    plt.title(titulo)
    plt.legend()
    plt.grid(True)

    plt.xticks(np.round(np.linspace(min(vols_pct), max(vols_pct), 6), 1),
               labels=[f'{x:.1f}%' for x in np.linspace(min(vols_pct), max(vols_pct), 6)])
    plt.yticks(np.round(np.linspace(min(retornos_pct), max(retornos_pct), 6), 1),
               labels=[f'{x:.1f}%' for x in np.linspace(min(retornos_pct), max(retornos_pct), 6)])

    plt.tight_layout()

    # 🔥 Aplica efeitos neon do cyberpunk
    mplcyberpunk.add_glow_effects()

    plt.show()




## Buscar ativos e preparar dados

In [ ]:
start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime.today()
# end_date = dt.datetime(2025, 10, 1)


tickers = [
           'PETR4.SA',
           'ITUB4.SA',
          #  'B3SA3.SA',
           'BPAC11.SA', 
           'WEGE3.SA',
           'ITSA4.SA',
           'MSFT34.SA',
           'NVDC34.SA',
         #   'VALE3.SA',
         #   'ELET3.SA',
        #    'AAPL34.SA',
        #    'M1TA34.SA',
           ]


prices = yf.download(tickers, start=start_date, end=end_date)['Close']
prices


### Matriz de covariância
A matriz de covariância (matrix_cov) mede como os retornos de diferentes ativos variam em relação uns aos outros.

Cada elemento da matriz representa a covariância entre dois ativos.

Valores positivos indicam que os ativos tendem a se mover na mesma direção, enquanto valores negativos indicam movimentos opostos.

#### Retorno logarítmico vs. aritmético
O retorno logarítmico é preferido em análises financeiras porque é simétrico e aditivo ao longo do tempo.

Isso significa que o retorno total de um período pode ser obtido somando os retornos logarítmicos diários.

Já o retorno aritmético (soma de %) não é aditivo devido à composição, o que pode levar a erros em análises de longo prazo.

**Exemplo:**
- Retorno aritmético: 10% + 10% = 20% (errado pois não considera a composição dos retornos dia a pós dia)
- Retorno logarítmico: ln(1+0.10) + ln(1+0.10) = ln(1.1) + ln(1.1) = 2*ln(1.1) = ln(1.1^2) = ln(1.21) = 21% 



In [ ]:
returns = prices.pct_change().apply(lambda x: np.log(1+x)).dropna() # Retorno logarítimo
returns_mean = returns.mean() # Media dos retornos diários
matrix_cov = returns.cov() # Matriz de covariância dos retornos diários

print("Média dos Retornos Diários:")
print(returns_mean)

print("\nMatriz de Covariância dos Retornos Diários:")
print(matrix_cov)

## Simulação de Carteiras Aleatórias

Este trecho de código simula 100.000 carteiras com diferentes combinações de pesos nos ativos, a fim de avaliar seu desempenho em termos de retorno, risco e índice de Sharpe. Abaixo está um resumo do que acontece:

- **Geração de pesos aleatórios:** Para cada carteira, são gerados pesos aleatórios, para cada ativo, que somam 1 (100% do capital investido).
- **Cálculo do retorno esperado anual:** O retorno esperado da carteira é calculado com base nos retornos médios diários dos ativos, multiplicado por 252 (dias úteis por ano).
- **Cálculo da volatilidade anual:** A volatilidade esperada da carteira é calculada a partir da matriz de covariância dos retornos diários, também anualizada.
- **Cálculo do índice de Sharpe:** Como a taxa livre de risco foi assumida como 0%, o índice de Sharpe é apenas o retorno anual dividido pela volatilidade anual.
- **Resultado:** Por fim, queremos a carteira com o maior índice de Sharpe, ou seja, a que oferece o melhor retorno esperado para o risco assumido.


A simulação gera os seguintes vetores:
- `vetor_retornos_esperados`: retorno anual esperado de cada carteira
- `vetor_volatilidades_esperadas`: volatilidade anual de cada carteira
- `vetor_sharpe`: índice de Sharpe de cada carteira
- `tabela_pesos`: pesos dos ativos em cada carteira simulada

A barra de progresso da `tqdm` mostra o andamento da simulação.


### Simulação base

In [ ]:
numero_carteiras = 100000 # Número total de carteiras simulados
vetor_retornos_esperados = np.zeros(numero_carteiras) # Vetor de retornos esperados 
vetor_volatilidades_esperadas = np.zeros(numero_carteiras) # Vetor de volatilidades esperadas
vetor_sharpe = np.zeros(numero_carteiras) # Vetor de índices de Sharpe
tabela_pesos = np.zeros((numero_carteiras, len(tickers))) # Tabela de pesos dos ativos

# Adiciona a barra de progresso ao loop
for k in tqdm(range(numero_carteiras), desc="Simulando carteiras"):
    pesos = np.random.random(len(tickers)) # Gera pesos aleatórios
    pesos = pesos/np.sum(pesos) # Normaliza os pesos para que a soma seja 1
    tabela_pesos[k, :] = pesos # Adiciona os pesos à tabela de pesos

    # Calcula o retorno esperado e a volatilidade esperada da carteira
    vetor_retornos_esperados[k] = np.sum(returns_mean * pesos) * 252 # Retorno esperado anualizado (252 dias úteis)
    # Calcula a volatilidade esperada da carteira
    vetor_volatilidades_esperadas[k] = np.sqrt(np.dot(pesos.T, np.dot(matrix_cov * 252, pesos))) # Volatilidade esperada anualizada

    # Calcula o índice de Sharpe (assumindo taxa livre de risco de 0%)
    vetor_sharpe[k] = vetor_retornos_esperados[k] / vetor_volatilidades_esperadas[k]

In [ ]:
np.set_printoptions(suppress=True)  # Desativa notação científica

# ===============================
# 🟢 Carteira com Sharpe Máximo (sem penalização)
# ===============================

max_sharpe_index = vetor_sharpe.argmax()  # Índice da carteira com maior índice de Sharpe
melhor_carteira = tabela_pesos[max_sharpe_index, :]  # Pesos da carteira ótima

print("\n\n🔝 Carteira com maior Índice de Sharpe (retorno máximo por risco):\n")

tabela_resultados = pd.DataFrame({
    'Ticker': tickers,
    'Peso (%)': [f'{peso*100:.2f}%' for peso in melhor_carteira]
})
print(tabela_resultados)

# Retorno esperado aritmético (usando log-retornos)
tabela_retornos_esperados_aritmetica = np.exp(vetor_retornos_esperados) - 1

print(f'\n📈 Retorno esperado (1 ano): {tabela_retornos_esperados_aritmetica[max_sharpe_index]*100:.2f}%')
print(f"📊 Índice de Sharpe: {vetor_sharpe[max_sharpe_index]:.4f}")

# ===============================
# 🔍 Carteira "vizinha" — próxima ao topo, mas com pesos diferentes
# ===============================

VIZINHO_N = 5

print("\n-----------------------------------------------------------------------------------------------------------")
print(f'\n🔄 Carteira {VIZINHO_N} da Vizinhança do Maior Índice de Sharpe:')

# Pega a 900ª melhor carteira no Sharpe tradicional
indices_ordenados = np.argsort(vetor_sharpe)
vizinhanca_max_sharpe_index = indices_ordenados[-(VIZINHO_N+1)]  # Índice da carteira vizinha
vizinhanca_carteira = tabela_pesos[vizinhanca_max_sharpe_index, :]

tabela_resultados_vizinha = pd.DataFrame({
    'Ticker': tickers,
    'Peso (%)': [f'{peso*100:.2f}%' for peso in vizinhanca_carteira]
})
print(tabela_resultados_vizinha)

print(f'\n📈 Retorno esperado (1 ano): {tabela_retornos_esperados_aritmetica[vizinhanca_max_sharpe_index]*100:.2f}%')
print(f"📊 Índice de Sharpe: {vetor_sharpe[vizinhanca_max_sharpe_index]:.4f}")

# ===============================
# 🧮 Top carteiras por Sharpe (tabela completa)
# ===============================

print("\n-----------------------------------------------------------------------------------------------------------")
df_top_sharpe = mostrar_top_carteiras(
    vetor_ordenacao=vetor_sharpe,
    vetor_retornos_esperados=vetor_retornos_esperados,
    tabela_pesos=tabela_pesos,
    tickers=tickers,
    top_n=2000,
    nome_ordenacao="Sharpe"
)

display(df_top_sharpe)

In [ ]:
# ===============================
# 📉 Gráfico da Fronteira Eficiente
# ===============================

VIZINHO_N = 5

vizinho_sharpe_index = indices_ordenados[-(VIZINHO_N+1)]

grafico_fronteira_eficiente = plot_fronteira_eficiente(
    volatilidades=vetor_volatilidades_esperadas,
    retornos=vetor_retornos_esperados,
    vetor_ordenacao=vetor_sharpe,
    titulo="Fronteira Eficiente - Sharpe (com Vizinhança)",
    label_ordenacao="Sharpe",
    cor_destaque="darkorange",
    idx_vizinho=vizinho_sharpe_index
)


display(grafico_fronteira_eficiente)


### Simulação penalizada por lambda

In [ ]:
# 🔧 Configuração penalização

lambda_penalizacao = 0.4  # Parâmetro que controla o peso da penalização

# ===============================================================================================

numero_carteiras = 100000  # Número total de carteiras simuladas
vetor_retornos_esperados = np.zeros(numero_carteiras)
vetor_volatilidades_esperadas = np.zeros(numero_carteiras)
vetor_sharpe = np.zeros(numero_carteiras)
vetor_sharpe_penalizado = np.zeros(numero_carteiras)  # Novo vetor
tabela_pesos = np.zeros((numero_carteiras, len(tickers)))

# Loop com barra de progresso
for k in tqdm(range(numero_carteiras), desc="Simulando carteiras"):
    pesos = np.random.random(len(tickers))
    pesos /= np.sum(pesos)  # Normaliza
    tabela_pesos[k, :] = pesos

    # Retorno e volatilidade anualizados
    retorno = np.sum(returns_mean * pesos) * 252
    volatilidade = np.sqrt(np.dot(pesos.T, np.dot(matrix_cov * 252, pesos)))

    vetor_retornos_esperados[k] = retorno
    vetor_volatilidades_esperadas[k] = volatilidade
    vetor_sharpe[k] = retorno / volatilidade

    # Penalidade por concentração (quanto mais concentrado, maior a penalidade)
    penalidade_concentracao = lambda_penalizacao * np.sum(pesos**2)
    vetor_sharpe_penalizado[k] = vetor_sharpe[k] - penalidade_concentracao


In [ ]:
np.set_printoptions(suppress=True)  # Desativa notação científica

# ===============================
# 🟢 Carteira com Sharpe Penalizado Máximo
# ===============================

max_sharpe_pen_index = vetor_sharpe_penalizado.argmax()  # Índice da carteira com maior Sharpe penalizado
melhor_carteira_penalizada = tabela_pesos[max_sharpe_pen_index, :]  # Pesos da carteira ótima penalizada

print("🔝 Carteira com maior Índice de Sharpe Penalizado (retorno ajustado à diversificação):\n")

tabela_resultados_pen = pd.DataFrame({
    'Ticker': tickers,
    'Peso (%)': [f'{peso*100:.2f}%' for peso in melhor_carteira_penalizada]
})
print(tabela_resultados_pen)

# Retorno esperado aritmético (usando log-retornos)
tabela_retornos_esperados_aritmetica = np.exp(vetor_retornos_esperados) - 1

print(f'\n📈 Retorno esperado (1 ano): {tabela_retornos_esperados_aritmetica[max_sharpe_pen_index]*100:.2f}%')
print(f"📊 Índice de Sharpe penalizado: {vetor_sharpe_penalizado[max_sharpe_pen_index]:.4f}")
print(f"📊 Sharpe sem penalização: {vetor_sharpe[max_sharpe_pen_index]:.4f}")

# ===============================
# 🔍 Carteira "vizinha" — Sharpe Penalizado
# ===============================
VIZINHO_N = 9

print("\n-----------------------------------------------------------------------------------------------------------")
print(f'\n🔄 Carteira {VIZINHO_N} da Vizinhança do Maior Índice de Sharpe Penalizado:')

indices_ordenados_pen = np.argsort(vetor_sharpe_penalizado)
vizinhanca_max_sharpe_index_pen = indices_ordenados_pen[-(VIZINHO_N+1)]
vizinhanca_carteira_pen = tabela_pesos[vizinhanca_max_sharpe_index_pen, :]

tabela_resultados_vizinha_pen = pd.DataFrame({
    'Ticker': tickers,
    'Peso (%)': [f'{peso*100:.2f}%' for peso in vizinhanca_carteira_pen]
})
print(tabela_resultados_vizinha_pen)

print(f'\n📈 Retorno esperado (1 ano): {tabela_retornos_esperados_aritmetica[vizinhanca_max_sharpe_index_pen]*100:.2f}%')
print(f"📊 Índice de Sharpe (sem penalização): {vetor_sharpe[vizinhanca_max_sharpe_index_pen]:.4f}")
print(f"📊 Índice de Sharpe penalizado: {vetor_sharpe_penalizado[vizinhanca_max_sharpe_index_pen]:.4f}")


# ===============================
# 🧮 Top carteiras por Sharpe (tabela completa)
# ===============================

print("\n-----------------------------------------------------------------------------------------------------------")
df_top_sharpe_penalizado = mostrar_top_carteiras(
    vetor_ordenacao=vetor_sharpe_penalizado,
    vetor_retornos_esperados=vetor_retornos_esperados,
    tabela_pesos=tabela_pesos,
    tickers=tickers,
    top_n=2000,
    nome_ordenacao="Sharpe Penalizado"
)

display(df_top_sharpe_penalizado)


In [ ]:
# ===============================
# 📉 Gráfico da Fronteira Eficiente com penalização
# ===============================

VIZINHO_N = 9

indices_ordenados_pen = np.argsort(vetor_sharpe_penalizado)  # ordenar pelo Sharpe penalizado
vizinho_sharpe_index_pen = indices_ordenados_pen[-(VIZINHO_N+1)]

grafico_fronteira_penalizada = plot_fronteira_eficiente(
    volatilidades=vetor_volatilidades_esperadas,
    retornos=vetor_retornos_esperados,
    vetor_ordenacao=vetor_sharpe_penalizado,
    titulo="Fronteira Eficiente",
    label_ordenacao="Sharpe Penalizado",
    cor_destaque="crimson",
    idx_vizinho=vizinho_sharpe_index_pen  # <<< AQUI
)


### Simulação com limitação de alocação

In [ ]:
# 🔧 Configuração de limites

peso_maximo = 0.25  # Limite único global

limitacao_por_ativo = False  # ➕ Troque para False se quiser usar limite único (ignora o global)
limites_por_ativo = np.array([0.3, 0.3, 0.25, 0.2, 0.5])  # Um por ativo (se usado)

# ===============================================================================================

numero_carteiras = 100000
vetor_retornos_esperados = np.zeros(numero_carteiras)
vetor_volatilidades_esperadas = np.zeros(numero_carteiras)
vetor_sharpe = np.zeros(numero_carteiras)
tabela_pesos = np.zeros((numero_carteiras, len(tickers)))

# Loop de simulação com restrições
for k in tqdm(range(numero_carteiras), desc="Simulando carteiras com restrições"):
    while True:
        pesos = np.random.random(len(tickers))
        pesos /= np.sum(pesos)

        # Aplica a lógica com base na flag
        if limitacao_por_ativo:
            if np.all(pesos <= limites_por_ativo):
                break
        else:
            if np.all(pesos <= peso_maximo):
                break

    tabela_pesos[k, :] = pesos

    retorno = np.sum(returns_mean * pesos) * 252
    volatilidade = np.sqrt(np.dot(pesos.T, np.dot(matrix_cov * 252, pesos)))

    vetor_retornos_esperados[k] = retorno
    vetor_volatilidades_esperadas[k] = volatilidade
    vetor_sharpe[k] = retorno / volatilidade


In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

np.set_printoptions(suppress=True)  # Desativa notação científica

# ===============================
# 🟢 Carteira com Sharpe Máximo (com restrição de 30%)
# ===============================

max_sharpe_index = vetor_sharpe.argmax()  # Índice da carteira com maior índice de Sharpe
melhor_carteira = tabela_pesos[max_sharpe_index, :]  # Pesos da carteira ótima

print("🔝 Carteira com maior Índice de Sharpe (com restrição de 30% por ativo):\n")

tabela_resultados = pd.DataFrame({
    'Ticker': tickers,
    'Peso (%)': [f'{peso*100:.2f}%' for peso in melhor_carteira]
})
print(tabela_resultados)

# Retorno esperado aritmético (usando log-retornos)
tabela_retornos_esperados_aritmetica = np.exp(vetor_retornos_esperados) - 1

print(f'\n📈 Retorno esperado (1 ano): {tabela_retornos_esperados_aritmetica[max_sharpe_index]*100:.2f}%')
print(f"📊 Índice de Sharpe: {vetor_sharpe[max_sharpe_index]:.4f}")

# ===============================
# 🔍 Carteira "vizinha" — próxima ao topo, mas com pesos diferentes
# ===============================

VIZINHO_N = 14

print("\n-----------------------------------------------------------------------------------------------------------")
print(f'\n🔄 Carteira {VIZINHO_N} da Vizinhança do Maior Índice de Sharpe:')

# Pega a 900ª melhor carteira no Sharpe tradicional
indices_ordenados = np.argsort(vetor_sharpe)
vizinhanca_max_sharpe_index = indices_ordenados[-(VIZINHO_N+1)]
vizinhanca_carteira = tabela_pesos[vizinhanca_max_sharpe_index, :]

tabela_resultados_vizinha = pd.DataFrame({
    'Ticker': tickers,
    'Peso (%)': [f'{peso*100:.2f}%' for peso in vizinhanca_carteira]
})
print(tabela_resultados_vizinha)

print(f'\n📈 Retorno esperado (1 ano): {tabela_retornos_esperados_aritmetica[vizinhanca_max_sharpe_index]*100:.2f}%')
print(f"📊 Índice de Sharpe: {vetor_sharpe[vizinhanca_max_sharpe_index]:.4f}")

# ===============================
# 🧮 Top carteiras por Sharpe (tabela completa)
# ===============================

print("\n-----------------------------------------------------------------------------------------------------------")
df_top_sharpe = mostrar_top_carteiras(
    vetor_ordenacao=vetor_sharpe,
    vetor_retornos_esperados=vetor_retornos_esperados,
    tabela_pesos=tabela_pesos,
    tickers=tickers,
    top_n=2000,
    nome_ordenacao="Sharpe"
)

display(df_top_sharpe)


In [ ]:
# ===============================
# 📉 Gráfico da Fronteira Eficiente com Restrições de Alocação
# ===============================

VIZINHO_N = 153

vizinho_sharpe_index = indices_ordenados[-(VIZINHO_N+1)]

grafico_fronteira_com_restricao = plot_fronteira_eficiente(
    volatilidades=vetor_volatilidades_esperadas,
    retornos=vetor_retornos_esperados,
    vetor_ordenacao=vetor_sharpe,
    titulo="Fronteira Eficiente - Carteiras com Restrições de Alocação",
    label_ordenacao="Sharpe",
    cor_destaque="seagreen",
    idx_vizinho=vizinho_sharpe_index
)

display(grafico_fronteira_com_restricao)


# Material Extra

### 🧠 Simulação de Carteiras com Penalização de Markowitz

Neste modelo de simulação, aplicamos o conceito de otimização de Markowitz, mas com uma **penalização** adicional para desincentivar carteiras muito **concentradas** em poucos ativos.  
A ideia é **buscar carteiras mais diversificadas**, adicionando uma penalidade proporcional à concentração dos pesos.

---

#### 🔹 Funcionamento da Simulação

- **1.** Geração de 100.000 carteiras aleatórias.
- **2.** Para cada carteira:
  - Calcula-se o **retorno esperado** anualizado.
  - Calcula-se a **volatilidade esperada** anualizada.
  - Calcula-se o **Índice de Sharpe** tradicional.
  - Aplica-se uma **penalização** proporcional à concentração dos pesos.

---


##### 1. Retorno Esperado da Carteira
$$
\mathbb{E}[R_p] = \sum_{i=1}^{N} w_i \cdot \mu_i \times 252
$$
Onde:
- \( w_i \) = peso do ativo \( i \)
- \( \mu_i \) = retorno diário médio do ativo \( i \)
- \( 252 \) = número de dias úteis no ano (para anualizar)

---

##### 2. Volatilidade Esperada da Carteira
$$
\sigma_p = \sqrt{w^T \cdot (\Sigma \times 252) \cdot w}
$$
Onde:
- \( \Sigma \) = matriz de covariância dos retornos diários
- \( w \) = vetor de pesos dos ativos
- \( 252 \) = número de dias úteis no ano

---

##### 3. Índice de Sharpe Tradicional
$$
\text{Sharpe} = \frac{\mathbb{E}[R_p]}{\sigma_p}
$$
*(Considerando taxa livre de risco igual a 0%.)*

---

##### 4. Penalização por Concentração
Para penalizar carteiras muito concentradas, utilizamos a **soma dos quadrados dos pesos** (\( \sum w_i^2 \)):

$$
\text{Penalidade} = \lambda \times \sum_{i=1}^{N} w_i^2
$$
Onde:
- \( \lambda \) é o **parâmetro de penalização** (definido como 0.3 no código).

---

##### 5. Sharpe Penalizado
A fórmula final usada para avaliar cada carteira é:

$$
\text{Sharpe Penalizado} = \text{Sharpe Tradicional} - \text{Penalidade}
$$

---

#### 📋 Intuição da Penalização

- **Se a carteira for muito concentrada** (pesos muito altos em poucos ativos), a penalidade será maior.
- **Se a carteira for bem diversificada** (pesos mais equilibrados), a penalidade será pequena.
- Assim, o algoritmo privilegia carteiras **com bom retorno por risco**, **mas também diversificadas**.

---

#### 📌 Conclusão

Este método gera carteiras eficientes não apenas do ponto de vista risco/retorno, mas também **mais seguras** contra concentração excessiva, o que reduz o risco específico de ativos individuais.

🚀 **Aplicando esta abordagem, você constrói portfólios mais robustos e resilientes!**


### Dica avançada:

O queremos mesmo é aperfeiçoar a maneira de projetar a média de retorno futuro (returns_mean).

Uma maneira mais avançada de fazer isso é usar a TIR (Taxa Interna de Retorno) do valuation.

Se você calculcar a TIR das empresas da bolsa através de um valuation decente, você consegue ter o retorno esperado de todas as empresas no futuro e consequentemente a volatilidade esperada também, a partir da volatilidade implícita utilizando opções.

**Contexto: Otimização de Markowitz**

- Na forma clássica da otimização de carteiras de Markowitz, usamos:

- A média histórica dos retornos para estimar o retorno esperado (𝐸[𝑅])

- A matriz de covariância histórica para representar o risco (volatilidade e correlação entre ativos)

- 💡 Mas usar média histórica pode ser muito fraco para prever o futuro — especialmente em mercados instáveis ou em empresas em transformação.

**1. TIR do Valuation como proxy para retorno esperado:**

- A TIR (Taxa Interna de Retorno) de um valuation é a taxa que igualaria o preço atual de uma ação com seus fluxos de caixa futuros esperados.

- Exemplo: Se a ação custa R$ 50 hoje e seu fluxo de caixa projetado indica que ela vai render R$ 60 em 1 ano, a TIR implícita é de 20%.

- Essa TIR pode ser interpretada como o retorno esperado daquele ativo no futuro.

- ✅ Isso dá uma estimativa mais fundamentada economicamente do que a simples média dos retornos passados.

**2. Volatilidade implícita (via opções) como proxy para risco:**

- Em vez de olhar a volatilidade histórica da ação, podemos usar a volatilidade implícita — que é extraída dos preços das opções no mercado.

- A volatilidade implícita reflete a expectativa do mercado quanto à variabilidade dos preços no futuro.

- ✅ Assim, você teria uma matriz de covariância mais condizente com as percepções futuras do mercado.

**3. Quando isso vale a pena?**

- Para investidores institucionais, gestores fundamentalistas ou modelos híbridos (quant + fundamental).

- Quando você tem boas ferramentas para valorar empresas e capturar precificação de opções.

- Em mercados onde o histórico recente não é confiável como proxy do futuro.
